# Database 2

In [1]:
import sqlite3
import pandas as pd

In [2]:
# ignore this cell (it's just to make certain text red later, but you don't need to understand it).
from IPython.core.display import HTML
HTML('<style>em { color: red; }</style>')

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
c = sqlite3.connect("movies.db")
c

In [5]:
pd.read_sql("select * from sqlite_master", c)

type    name tbl_name  rootpage  \
0  table  movies   movies         2   

                                                 sql  
0  CREATE TABLE "movies" (\n"Title" TEXT,\n  "Dir...

In [6]:
pd.read_sql("select * from movies", c).head(15)

Title              Director  Year  \
0                   Guardians of the Galaxy            James Gunn  2014   
1                                Prometheus          Ridley Scott  2012   
2                                     Split    M. Night Shyamalan  2016   
3                                      Sing  Christophe Lourdelet  2016   
4                             Suicide Squad            David Ayer  2016   
5                            The Great Wall           Yimou Zhang  2016   
6                                La La Land       Damien Chazelle  2016   
7                                  Mindhorn            Sean Foley  2016   
8                        The Lost City of Z            James Gray  2016   
9                                Passengers         Morten Tyldum  2016   
10  Fantastic Beasts and Where to Find Them           David Yates  2016   
11                           Hidden Figures        Theodore Melfi  2016   
12                                Rogue One        Gareth Edwards  2016   
13                                    Moana          Ron Clements  2016   
14                                 Colossal       Nacho Vigalondo  2016   

    Runtime  Rating  Revenue  
0       121     8.1   333.13  
1       124     7.0   126.46  
2       117     7.3   138.12  
3       108     7.2   270.32  
4       123     6.2   325.02  
5       103     6.1    45.13  
6       128     8.3   151.06  
7        89     6.4     0.00  
8       141     7.1     8.01  
9       116     7.0   100.01  
10      133     7.5   234.02  
11      127     7.8   169.27  
12      133     7.9   532.17  
13      107     7.7   248.75  
14      109     6.4     2.87

In [8]:
def qry(sql, cap=10):
    return pd.read_sql(sql, c).head(cap)

In [10]:
qry("""
SELECT *
FROM movies
""", 3)

Title            Director  Year  Runtime  Rating  Revenue
0  Guardians of the Galaxy          James Gunn  2014      121     8.1   333.13
1               Prometheus        Ridley Scott  2012      124     7.0   126.46
2                    Split  M. Night Shyamalan  2016      117     7.3   138.12

# Review: Simple Selections

### Which *movie* has the *highest rating*?

In [14]:
qry("""
SELECT title, rating
FROM movies
ORDER BY rating DESC
LIMIT 1
""")

Title  Rating
0  The Dark Knight     9.0

In [15]:
qry("""
SELECT *
FROM movies
""", 3)

Title            Director  Year  Runtime  Rating  Revenue
0  Guardians of the Galaxy          James Gunn  2014      121     8.1   333.13
1               Prometheus        Ridley Scott  2012      124     7.0   126.46
2                    Split  M. Night Shyamalan  2016      117     7.3   138.12

### Which *director* made the *shortest movie*?

In [17]:
qry("""
SELECT director, runtime
FROM movies
ORDER BY Runtime ASC
LIMIT 1
""")

Director  Runtime
0  Claude Barras       66

### Which *director* made the *highest-revenue movie*?

### Which *movie* had the *highest revenues* in *2016*?

### Which *3 movies* had the *highest revenues* in *2016*?

In [21]:
qry("""
SELECT title, revenue, year
FROM movies
WHERE year = 2016
ORDER BY revenue DESC
LIMIT 3
""")

Title  Revenue  Year
0                   Rogue One   532.17  2016
1                Finding Dory   486.29  2016
2  Captain America: Civil War   408.08  2016

### Which *3 movies* have the *highest rating-to-revenue ratios*?

Introduce `AS`

In [24]:
qry("""
SELECT title, rating/revenue AS ratio
FROM movies
ORDER BY ratio DESC
LIMIT 3
""")

Title  ratio
0    Wakefield  750.0
1  Love, Rosie  720.0
2     Lovesong  640.0

# Aggregate Queries

```
SUM, AVG, COUNT, MIN, MAX
```

### How many *movies* are there?

In [29]:
qry("""
SELECT COUNT(*)
FROM movies
""")

COUNT(*)
0       998

### How many *directors* are there?

In [28]:
qry("""
SELECT COUNT(DISTINCT director)
FROM movies
""")

COUNT(DISTINCT director)
0                       643

### What is the *total revenue* of *all the movies*?

In [30]:
qry("""
SELECT SUM(revenue)
FROM movies
""")

SUM(revenue)
0      72215.45

### What is the *average rating* across *all movies*?

* v1: with `SUM` and `COUNT`
* v2: with `AVG`

In [32]:
qry("""
SELECT SUM(rating)/COUNT(*) AS avg_rating
FROM movies
""")

avg_rating
0    6.723447

In [33]:
qry("""
SELECT AVG(rating)
FROM movies
""")

AVG(rating)
0     6.723447

### What is the *average revenue* and *average runtime* of *all the movies*?

In [34]:
qry("""
SELECT AVG(revenue), AVG(runtime)
FROM movies
""")

AVG(revenue)  AVG(runtime)
0      72.36017    113.170341

### What is the *average runtime* for a *James Gunn* movie?

In [35]:
qry("""
SELECT AVG(runtime)
FROM movies
WHERE director = "James Gunn"
""")

AVG(runtime)
0         104.0

### What is the *average revenue* for a *Ridley Scott* movie?

### *How many movies* were there in *2016*?

### What *percentage* of the *total revenue* came from the *highest-revenue movie*?

### What *percentage* of the *total revenue* came from the *highest-revenue movie* in *2016*?

In [36]:
qry("""
SELECT 100*MAX(revenue)/SUM(revenue) as percent
FROM movies
WHERE year = 2016
""")

percent
0  4.746581

In [37]:
qry("""
SELECT *
FROM movies
WHERE year = 2016
ORDER BY revenue DESC
LIMIT 1
""")

Title        Director  Year  Runtime  Rating  Revenue
0  Rogue One  Gareth Edwards  2016      133     7.9   532.17

# GROUP BY Queries

```sql
SELECT ???, ??? FROM Movies
GROUP BY ???
```

### What is the *total revenue* per each *year*?

* v1: the amounts
* v2: the amounts, as labeled by year

In [38]:
qry("""
SELECT SUM(revenue)
FROM movies
GROUP BY year
""")

SUM(revenue)
0       3624.46
1       4306.23
2       5053.22
3       5292.26
4       5989.65
5       5431.96
6       6910.29
7       7544.21
8       7997.40
9       8854.12

In [41]:
qry("""
SELECT year, SUM(revenue)
FROM movies
GROUP BY year
""")

Year  SUM(revenue)
0  2006       3624.46
1  2007       4306.23
2  2008       5053.22
3  2009       5292.26
4  2010       5989.65
5  2011       5431.96
6  2012       6910.29
7  2013       7544.21
8  2014       7997.40
9  2015       8854.12

### *How many movies* were by each *director*?

In [43]:
qry("""
SELECT director, COUNT(*), AVG(rating)
FROM movies
GROUP BY director
""")

Director  COUNT(*)  AVG(rating)
0           Aamir Khan         1          8.5
1  Abdellatif Kechiche         1          7.8
2            Adam Leon         1          6.5
3           Adam McKay         4          7.0
4        Adam Shankman         2          6.3
5         Adam Wingard         2          5.9
6        Afonso Poyart         1          6.4
7        Aisling Walsh         1          7.8
8         Akan Satayev         1          6.3
9       Akiva Schaffer         1          6.7

### What is the *average rating* for each *director*?

### What is the *average runtime* for each *director*?

### How many *unique directors* created a movie in each *year*

In [45]:
qry("""
SELECT director, COUNT(distinct year)
FROM movies
GROUP BY director
""")

Director  COUNT(distinct year)
0           Aamir Khan                     1
1  Abdellatif Kechiche                     1
2            Adam Leon                     1
3           Adam McKay                     4
4        Adam Shankman                     2
5         Adam Wingard                     2
6        Afonso Poyart                     1
7        Aisling Walsh                     1
8         Akan Satayev                     1
9       Akiva Schaffer                     1

# Combining GROUP BY with other CLAUSES

<img src="groupby.png">

### What is the *total revenue* of per *year*, in *recent* years?

### Which *directors* have had the *largest number of movies* earning *over 100M dollars*?

In [47]:
qry("""
SELECT director, COUNT(*) AS mov_count
FROM movies
WHERE revenue > 100
GROUP BY director
ORDER BY mov_count DESC
""")

Director  mov_count
0        David Yates          6
1        J.J. Abrams          5
2  Christopher Nolan          4
3       Dennis Dugan          4
4   Francis Lawrence          4
5         Justin Lin          4
6        Michael Bay          4
7          Paul Feig          4
8       Ridley Scott          4
9        Zack Snyder          4

### Which *three* of the *directors* have the *greatest average rating*?

In [51]:
qry("""
SELECT director, AVG(rating) AS avg_rating, COUNT(*) as mov_count
FROM movies
GROUP BY director
ORDER BY avg_rating DESC
LIMIT 3
""")

Director  avg_rating  mov_count
0      Nitesh Tiwari        8.80          1
1  Christopher Nolan        8.68          5
2     Makoto Shinkai        8.60          1

Why is the above question maybe not the best to ask?

### Which *three* of the *directors* have the *greatest average rating* over at *least three movies*?

Need filtering BEFORE and AFTER the GROUP operations

<img src="pipeline.png">

# WHERE vs. HAVING

* WHERE: filter rows in original table
* HAVING: filter groups

### Repeat: Which *three* of the *directors* have the *greatest average rating* over at *least three movies*?

<img src="having.png">

In [52]:
qry("""
SELECT director, AVG(rating) AS avg_rating, COUNT(*) as mov_count
FROM movies
GROUP BY director
HAVING mov_count >= 3
ORDER BY avg_rating DESC
LIMIT 3
""")

Director  avg_rating  mov_count
0  Christopher Nolan        8.68          5
1    Martin Scorsese        7.92          5
2  Quentin Tarantino        7.90          4

### Which *directors* have had *more than 2 movies* that have been *since 2014*?

In [57]:
qry("""
SELECT director, COUNT(*) as mov_count
FROM movies
WHERE year > 2013
GROUP BY director
HAVING mov_count > 2
""")

Director  mov_count
0     Antoine Fuqua          3
1     Mike Flanagan          3
2  Nicholas Stoller          3

### Which *directors* have had more than *five* movies with runtimes under *100* minutes

In [ ]:
c.close()